In [1]:
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
from ckiptagger import construct_dictionary
import pandas as pd

host = 4
host = 0

if host == 0:
    path = r'/Users/aron/Documents/GitHub/Perfume/2_NLP'
    dict_file = r'/Users/aron/Documents/GitHub/Perfume/1_Crawler/Resource/dict.xlsx'
    
elif host == 4:
    path = r'/home/rserver/Data_Mining/personal_workspace/yz/Lab/CkipTagger/'
    dict_file = r'/Users/aron/Documents/GitHub/Perfume/1_Crawler/Resource/dict.xlsx'
    
path_resource = path + '/Resource'
path_export = path + '/Export'

## Import Custom Dictionary

In [2]:
dict_df = pd.read_excel(dict_file)
dict_li = dict_df['word'].tolist()
custom_dict = dict((el, 1) for el in dict_li)
custom_dict = construct_dictionary(custom_dict)

In [4]:
# 2. Load model
# To use GPU:
#    1. Install tensorflow-gpu (see Installation)
#    2. Set CUDA_VISIBLE_DEVICES environment variable, e.g. os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#    3. Set disable_cuda=False, e.g. ws = WS("./data", disable_cuda=False)
# To use CPU:
# https://drive.google.com/drive/folders/105IKCb88evUyLKlLondvDBoh7Dy_I1tm

ws = WS(path_resource + "/data")
pos = POS(path_resource + "/data")
ner = NER(path_resource + "/data")

/Users/aron/opt/anaconda3/lib/python3.8/site-packages/ckiptagger/model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
/Users/aron/opt/anaconda3/lib/python3.8/site-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:984: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self._kernel = self.add_variable(
/Users/aron/opt/anaconda3/lib/python3.8/site-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:993: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self._bias = self.add_variable(
/Users/aron/opt/anaconda3/lib/python3.8/site-packages/ckiptagger/model_pos.py:56: UserWarning: `tf.nn.rnn_cell.LSTMC

In [ ]:
# 3. (Optional) Create dictionary
# You can supply words for WS special consideration, including their relative weights.
# word_to_weight = {
#     "土地公": 1,
#     "土地婆": 1,
#     "公有": 2,
#     "": 1,
#     "來亂的": "啦",
#     "緯來體育台": 1,
# }
# dictionary = construct_dictionary(word_to_weight)
# print(dictionary)

## 4. Run the WS-POS-NER pipeline

In [ ]:
data = pd.read_excel(path_resource + '/data.xlsx')
data = data.dropna(subset=['name'], axis=0)
sentence_list = data['name'].tolist()

In [ ]:
word_sentence_list = ws(
    sentence_list,
    # sentence_segmentation = True, # To consider delimiters
    # segment_delimiter_set = {",", "。", ":", "?", "!", ";"}), # This is the defualt set of delimiters
    # recommend_dictionary = dictionary1, # words in this dictionary are encouraged
    coerce_dictionary = custom_dict, # words in this dictionary are forced
)

pos_sentence_list = pos(word_sentence_list)
entity_sentence_list = ner(word_sentence_list, pos_sentence_list)

In [ ]:
# 5. (Optional) Release memory
# del ws
# del pos
# del ner

In [ ]:
# 6. Show Results
def print_word_pos_sentence(word_sentence, pos_sentence):
    assert len(word_sentence) == len(pos_sentence)
    for word, pos in zip(word_sentence, pos_sentence):
        print(f"{word}({pos})", end="\u3000")
    print()
    return


In [ ]:
len(sentence_list)

In [ ]:
for i, sentence in enumerate(sentence_list):
#     print()
    print(f"'{sentence}'")
    print_word_pos_sentence(word_sentence_list[i],  pos_sentence_list[i])
    for entity in sorted(entity_sentence_list[i]):
        print(entity)

In [ ]:
entity

## Feature Extraction

In [ ]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s['Word'].values.tolist(), 
                                                           s['POS'].values.tolist(), 
                                                           s['Tag'].values.tolist())]
        self.grouped = self.data.groupby('Sentence #').apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None
getter = SentenceGetter(df)
sentences = getter.sentences

In [ ]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True
        
    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]
def sent2labels(sent):
    return [label for token, postag, label in sent]
def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [ ]:


X = [sent2features(s) for s in sentence_list]